In [56]:
import pandas as pd
import numpy as np
from scipy.interpolate import interp1d
import scipy.integrate as integrate

In [128]:
hazard_rates = pd.read_csv('hazard_rates.csv')
hazard_rates.sort_values(by = 'SettlementDate', inplace=True)

discount_factors = pd.read_csv('discount_factor_common_dates.csv')
discount_factors = discount_factors.set_index('Date')
dfc = discount_factors.columns
discount_factors['zero'] = 1
discount_factors = discount_factors[np.append('zero', dfc )]
discount_factors.sort_values(by = 'Date', inplace=True)
discount_factors.reset_index(inplace=True)

coupons = pd.read_excel('treasury_bond_note_coupons.xlsx')

cds = pd.read_csv('cds_data_processed.csv')
cds.sort_values(by = 'Settlement Date', inplace=True)


In [129]:
mean_coupons = coupons.groupby('Tenor').mean().reset_index()

In [130]:
cds['Open_6m'] = cds['Open_6m']/100
cds['Open_1yr'] = cds['Open_1yr']/100
cds['Open_2yr'] = mean_coupons[mean_coupons['Tenor'] == 2]['Cpn'].values[0] - cds['Open_2yr']/100
cds['Open_3yr'] = mean_coupons[mean_coupons['Tenor'] == 3]['Cpn'].values[0] - cds['Open_3yr']/100
cds['Open_3yr'] = mean_coupons[mean_coupons['Tenor'] == 5]['Cpn'].values[0] - cds['Open_5yr']/100
cds['Open_3yr'] = mean_coupons[mean_coupons['Tenor'] == 7]['Cpn'].values[0] - cds['Open_7yr']/100
cds['Open_3yr'] = mean_coupons[mean_coupons['Tenor'] == 10]['Cpn'].values[0] - cds['Open_10yr']/100

In [131]:
time_periods_m = np.array([0, 1/12,3/12, 6/12, 9/12, 1])
time_periods_y = np.arange(2, 31, 1)
time_periods = np.append(time_periods_m, time_periods_y)


In [114]:
discount_factors_funcs = discount_factors.apply(lambda row: interp1d(time_periods, row[1:], kind='cubic'), axis = 1)

In [140]:
cds_time_periods = [0.5, 1, 2, 3, 5, 7, 10]

In [ ]:
discount_factors_subset_times = discount_factors.iloc[:, [4, 6, 7, 8,10,12,15]]
cds_subset_times = cds.iloc[:, 8:]

In [213]:
df_integrate = np.empty(shape =(cds.shape[0],len(cds_time_periods )))
for i in range(discount_factors.shape[0]):
    for j, val in enumerate(cds_time_periods):
        df_integrate[i, j]  = integrate.quad(lambda x: discount_factors_funcs[i](x), 0,val)[0]

In [ ]:
hz_moded = hazard_rates.copy()
hz_moded

In [ ]:
hz_moded['Maturity_6m'] = np.exp(-hz_moded['Maturity_6m']*0.5)
hz_moded['Maturity_1yr'] = np.exp(-hz_moded['Maturity_1yr']*1)
hz_moded['Maturity_2yr'] = np.exp(-hz_moded['Maturity_1yr']*2)
hz_moded['Maturity_3yr'] = np.exp(-hz_moded['Maturity_1yr']*3)
hz_moded['Maturity_5yr'] = np.exp(-hz_moded['Maturity_1yr']*5)
hz_moded['Maturity_7yr'] = np.exp(-hz_moded['Maturity_1yr']*7)
hz_moded['Maturity_10yr'] = np.exp(-hz_moded['Maturity_1yr']*10)

In [222]:
np.multiply(np.add(np.multiply(cds_subset_times, df_integrate), discount_factors_subset_times)

/var/folders/34/gh568vr946d45y78dns07shm0000gn/T/ipykernel_44917/2211659236.py:1: FutureWarning: Calling a ufunc on non-aligned DataFrames (or DataFrame/Series combination). Currently, the indices are ignored and the result takes the index/columns of the first DataFrame. In the future , the DataFrames/Series will be aligned before applying the ufunc.
Convert one of the arguments to a NumPy array (eg 'ufunc(df1, np.asarray(df2)') to keep the current behaviour, or align manually (eg 'df1, df2 = df1.align(df2)') before passing to the ufunc to obtain the future behaviour and silence this warning.
  np.add(np.multiply(cds_subset_times, df_integrate), discount_factors_subset_times)


,Open_6m,Open_1yr,Open_2yr,Open_3yr,Open_5yr,Open_7yr,Open_10yr
3158,1.126259,1.250217,5.947292,16.947344,192.344265,266.519217,366.783740
3157,1.135749,1.269424,5.903083,16.925962,207.085388,280.770741,382.860178
3156,1.121486,1.240753,5.977147,17.047418,185.762396,254.026040,347.895764
3155,1.117320,1.232371,5.997402,17.106560,179.725228,243.082867,331.167939
3154,1.108200,1.214095,6.041823,17.151555,166.326833,228.439510,313.218665
...,...,...,...,...,...,...,...
4,1.033987,1.072658,6.317920,17.333446,78.383604,126.098715,193.911966
3,1.034026,1.072488,6.314333,17.309134,78.369760,125.664785,196.767042
2,1.034229,1.073027,6.313309,17.316857,78.701003,126.641440,195.069426
1,1.034114,1.072960,6.317532,17.331900,78.821580,126.908459,195.712704


,SettlementDate,Maturity_6m,Maturity_1yr,Maturity_2yr,Maturity_3yr,Maturity_5yr,Maturity_7yr,Maturity_10yr
2957,01-Apr-2010,0.997844,0.004317,0.006977,0.007081,0.007925,0.008196,0.008193
2713,01-Apr-2011,0.998046,0.003912,0.005806,0.006963,0.009000,0.011829,0.013391
2258,01-Apr-2013,0.997952,0.004133,0.004217,0.005222,0.009040,0.013991,0.014324
2012,01-Apr-2014,0.999421,0.001159,0.001432,0.002281,0.004736,0.007918,0.008201
1763,01-Apr-2015,0.998551,0.003056,0.003176,0.002836,0.003213,0.007924,0.007996
...,...,...,...,...,...,...,...,...
1866,31-Oct-2014,0.998657,0.002578,0.002569,0.002882,0.003253,0.006692,0.007481
1373,31-Oct-2016,0.999384,0.001717,0.003801,0.004969,0.005952,0.007022,0.008972
1120,31-Oct-2017,0.999352,0.001746,0.003612,0.004668,0.005443,0.007094,0.009422
869,31-Oct-2018,0.998980,0.002282,0.002430,0.003567,0.004635,0.005143,0.006135
